In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier 
import matplotlib.pyplot as plt


# Load the TCR sequence data
df = pd.read_csv('vdjdb_excel.csv')

# Show unique species
print("Unique species:", df['species'].unique())

# Define the key features and the target variable
key_features = ["cdr3", 'v.segm', 'j.segm', 'species', "mhc.a"]
target = 'antigen.epitope'

# Drop rows with missing values in the relevant columns
df.dropna(subset=key_features + [target], inplace=True)

# Create a new DataFrame containing only the key features for analysis
df_key_features = df[key_features].copy()

# Convert the target variable into categorical codes for machine learning models
df_target = df[target].astype('category').cat.codes

# Encode all key features as categorical codes to prepare for machine learning
for column in df_key_features.columns:
    df_key_features[column] = df_key_features[column].astype('category').cat.codes

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_key_features, df_target, test_size=0.2, random_state=42)


print(df_key_features.head())
print(df_target.head())


print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Unique species: ['HomoSapiens' 'MusMusculus' 'MacacaMulatta']
    cdr3  v.segm  j.segm  species  mhc.a
0  60524      60      36        0     39
1  45957     157      60        0     39
2  21030     157      60        0     39
3  54282      53      19        0     39
4  21061     157      60        0     39
0    198
1    198
2    198
3    198
4    198
dtype: int16
Training set size: (73289, 5)
Test set size: (18323, 5)


In [7]:
# Initialize the baseline model
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
baseline_pred = dummy.predict(X_test)
baseline_accuracy = accuracy_score(y_test, baseline_pred)
print(f"Baseline Model Accuracy: {baseline_accuracy:}") 


# Define the parameter grid for RandomForest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15],
    'min_samples_split': [5, 20],
    'min_samples_leaf': [4, 8]
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Use KFold for cross-validation
kf = KFold(n_splits=2, shuffle=True, random_state=42)

# Initialize GridSearchCV with KFold
grid_search = GridSearchCV(rf, param_grid, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Use the best estimator to make predictions on the training and test data
y_pred_train = grid_search.best_estimator_.predict(X_train)
y_pred_test = grid_search.best_estimator_.predict(X_test)

# Calculate and print the accuracy on the training and test data
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"Training Accuracy: {accuracy_train}") 
print(f"Test Accuracy: {accuracy_test}")  



Baseline Model Accuracy: 0.3025705397587731
Fitting 2 folds for each of 16 candidates, totalling 32 fits
Best Parameters: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 200}
Best Cross-Validation Score: 0.7000095504281447
Training Accuracy: 0.7584767154688971
Test Accuracy: 0.7161491022212519
